In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals
from __future__ import print_function

In [16]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from make_pairwise_gold_metric_scores import compute_metrics

pd.set_option('max_colwidth',5000)

In [17]:
# Load word2vec model for this specific course
w2v_matrix = pickle.load( open( "../data/w2v_matrix_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) )
vocab = np.array(pickle.load( open( "../data/vocab_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) ) )

In [18]:
from sklearn.decomposition import PCA

In [19]:
def tfidf_lsa(gold_data, gold_matrix, eigenvector=20):
  # Get tfidf counts for each comment as a matrix C with shape (# comments, size of vocab)
  tfidf = TfidfVectorizer(stop_words='english')
  C = tfidf.fit_transform(gold_data.body.values)
  print("tfidf C shape", C.shape)
  lsa = TruncatedSVD(eigenvector, algorithm = 'arpack')
  A = lsa.fit_transform(C)
  print("LSA shape", A.shape)
  A=normalize(A,norm="l2")
  # We compute pairwise cosine similarity with dot product since A is normalized.
  pairwise_cosine_similarity = np.dot(A, A.transpose())
  return MinMaxScaler().fit_transform(pairwise_cosine_similarity)

In [20]:
# Load gold train data
gold_matrix_train = pd.read_csv('gold_matrix_train_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
df_gold_train = pd.read_csv('gold_data_train_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
pairwise_cosine_similarity_train = tfidf_lsa(df_gold_train, gold_matrix_train)

tfidf C shape (761, 6942)
LSA shape (761, 20)


In [21]:
# Load gold data
gold_matrix_test = pd.read_csv('gold_matrix_test_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
df_gold_test = pd.read_csv('gold_data_test_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
pairwise_cosine_similarity_test = tfidf_lsa(df_gold_test, gold_matrix_test)

tfidf C shape (649, 6583)
LSA shape (649, 20)


In [22]:
metrics = compute_metrics(pairwise_cosine_similarity_train, pairwise_cosine_similarity_test, gold_matrix_train, df_gold_train, gold_matrix_test, df_gold_test)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.863219697959 - 0.431818775359 = 0.4314009226
Pairwise Binary Logistic Regression Accuracy score: 0.867210666641

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.867039726876


,Score
logreg_acc_pairwise_binary,0.867211
median_quantile_diff,0.431401
random_forest_acc_pairwise_binary,0.867040


In [23]:
# Switching train and test

metrics = compute_metrics(pairwise_cosine_similarity_test, pairwise_cosine_similarity_train, gold_matrix_test, df_gold_test, gold_matrix_train, df_gold_train)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.828400282497 - 0.423038535988 = 0.405361746509
Pairwise Binary Logistic Regression Accuracy score: 0.836224208758

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.836189673661


,Score
logreg_acc_pairwise_binary,0.836224
median_quantile_diff,0.405362
random_forest_acc_pairwise_binary,0.836190
